In [1]:
import numpy as np

In [2]:
import onnx

In [3]:
import os

In [4]:
import glob

In [5]:
from onnx import numpy_helper

In [6]:
import tvm
from tvm import te
import tvm.relay as relay

In [7]:
mnist_model = onnx.load('../mnist/mnist-8.onnx')

In [8]:
test_data_dir = '../mnist/test_data_set_2'

In [9]:
## Load inputs

In [10]:
inputs = []

In [11]:
inputs_num = len(glob.glob(os.path.join(test_data_dir, 'input_*.pb')))

In [12]:
#print(inputs_num)

In [13]:
for i in range(inputs_num):
    input_file = os.path.join(test_data_dir, 'input_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    with open(input_file, 'rb') as f:
        tensor.ParseFromString(f.read())
    #inputs.append(numpy_helper.to_array(tensor))
    input_array = numpy_helper.to_array(tensor)

In [14]:
#input_array = np.array(inputs)
print(input_array.shape)

(1, 1, 28, 28)


In [15]:
# Load reference outputs

In [16]:
ref_outputs = []

In [17]:
ref_outputs_num = len(glob.glob(os.path.join(test_data_dir, 'output_*.pb')))

In [18]:
for i in range(ref_outputs_num):
    output_file = os.path.join(test_data_dir, 'output_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    with open(output_file, 'rb') as f:
        tensor.ParseFromString(f.read())
    ref_outputs.append(numpy_helper.to_array(tensor))

In [19]:
print(ref_outputs)

[array([[-2334.0889  , -1113.6254  ,  1076.5801  ,  -860.2391  ,
         1588.535   , -1534.347   , -2686.0776  ,   -56.480476,
           74.57547 ,  3715.3801  ]], dtype=float32)]


In [20]:
# tvm compile

In [21]:
input_name = "Input3"

In [22]:
shape_dict = {input_name: input_array.shape}
print(shape_dict)

{'Input3': (1, 1, 28, 28)}


In [23]:
mod, params = relay.frontend.from_onnx(mnist_model, shape_dict)

In [24]:
# optianal

In [25]:
target = "llvm"

In [26]:
with tvm.transform.PassContext(opt_level=3):
    intrp = relay.build_module.create_executor("graph", mod, tvm.cpu(0), target)

In [27]:
print(intrp)

In [28]:
dtype = "float32"
tvm_output = intrp.evaluate()(tvm.nd.array(input_array.astype(dtype)), **params).asnumpy()

Cannot find config for target=llvm -keys=cpu -link-params=0, workload=('dense_pack.x86', ('TENSOR', (1, 256), 'float32'), ('TENSOR', (10, 256), 'float32'), None, 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm -keys=cpu -link-params=0, workload=('conv2d_NCHWc.x86', ('TENSOR', (1, 8, 18, 18), 'float32'), ('TENSOR', (16, 8, 5, 5), 'float32'), (1, 1), (0, 0, 0, 0), (1, 1), 'NCHW', 'NCHW', 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm -keys=cpu -link-params=0, workload=('conv2d_NCHWc.x86', ('TENSOR', (1, 1, 32, 32), 'float32'), ('TENSOR', (8, 1, 5, 5), 'float32'), (1, 1), (0, 0, 0, 0), (1, 1), 'NCHW', 'NCHW', 'float32'). A fallback configuration is used, which may bring great performance regression.


In [29]:
print(tvm_output)

[[-2334.0884   -1113.6259    1076.5802    -860.23926   1588.5348
  -1534.3464   -2686.0781     -56.480286    74.574875  3715.3796  ]]
